<a href="https://colab.research.google.com/github/Niko-k98/Machine-Learning-class/blob/main/Q5_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import numpy as np
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
import pandas


In [4]:
# Load and clean a text file
def fClean_Load(filename):
    file = open(filename, encoding="utf8", errors='ignore') 
    #file = open(filename, 'rb')
    text = file.read()
    file.close()
    # Clean text
    words = re.findall(r'[a-z\.]+', text.lower())
    return ' '.join(words)


In [7]:
# load text / Complete novel "A Tale of Two Cities"
raw_text = fClean_Load('AToTC.txt')

In [8]:
# organize into sequences of characters

length = 100
lines = list()
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i+1]
    lines.append(seq)
print('Total lines: %d' % len(lines))


Total lines: 729489


In [9]:
chars = sorted(list(set(''.join(lines))))
mapping = dict((c, i) for i, c in enumerate(chars))


In [10]:
sequences = list()
for line in lines:
	encoded_seq = [mapping[char] for char in line]
	sequences.append(encoded_seq)

In [11]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 28


In [12]:
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [13]:
#X = X / float(vocab_size)
#X = np.reshape(X, (X.shape[0], length, 1))
#y = to_categorical(y, num_classes=vocab_size)
sequences = np.array(sequences)
X1, y = sequences[:,:-1], sequences[:,-1]
temp = [to_categorical(x, num_classes=vocab_size) for x in X1]
X = np.array(temp)
y = to_categorical(y, num_classes=vocab_size)

In [23]:
##############################################################################
####################### Select and fit an appropriate model ##################
# 1) LSTM size, 2) Dropout, 3) epochs, and 4) batch_size #####################
##############################################################################

model = Sequential()
model.add(LSTM(1024  ,input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 1024)              4313088   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 28)                28700     
Total params: 4,341,788
Trainable params: 4,341,788
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
history=model.fit(X, y, epochs=50, verbose=1, batch_size=128 )


Epoch 1/50
5700/5700 [==============================] - 365s 64ms/step - loss: 1.4067 - accuracy: 0.5642
Epoch 2/50
5700/5700 [==============================] - 364s 64ms/step - loss: 1.2897 - accuracy: 0.5963
Epoch 3/50
5700/5700 [==============================] - 364s 64ms/step - loss: 1.2251 - accuracy: 0.6145
Epoch 4/50
5700/5700 [==============================] - 364s 64ms/step - loss: 1.1743 - accuracy: 0.6282
Epoch 5/50
5700/5700 [==============================] - 363s 64ms/step - loss: 1.1352 - accuracy: 0.6397
Epoch 6/50
5700/5700 [==============================] - 364s 64ms/step - loss: 1.1000 - accuracy: 0.6494
Epoch 7/50
5700/5700 [==============================] - 363s 64ms/step - loss: 1.0689 - accuracy: 0.6582
Epoch 8/50
5700/5700 [==============================] - 363s 64ms/step - loss: 1.0410 - accuracy: 0.6664
Epoch 9/50
5700/5700 [==============================] - 363s 64ms/step - loss: 1.0161 - accuracy: 0.6740
Epoch 10/50
5700/5700 [==============================] 

In [32]:
pandas.DataFrame(history.history).to_csv("history.csv")

In [27]:
# Save and test using code from the Q4_Test
model.save('LargeLSTM_model.h5')
dump(mapping, open('LargeLSTM_mapping.pkl', 'wb'))


In [33]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [34]:
import string as str
seed_text = 'It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair,we had everything before us, we had nothing before us, we were all going direct to Heaven, we were all going direct the other way— '
seed_text = seed_text.lower()
seed_text = "".join(c for c in seed_text if c not in ('!','.',':',',',';','-','—'))
n_chars_to_predict = 500
seq_length = 20
print(seed_text)


it was the best of times it was the worst of times it was the age of wisdom it was the age of foolishness it was the epoch of belief it was the epoch of incredulity it was the season of light it was the season of darkness it was the spring of hope it was the winter of despairwe had everything before us we had nothing before us we were all going direct to heaven we were all going direct the other way 


In [35]:
# load the model and mapping
model = load_model('LargeLSTM_model.h5')
mapping = load(open('LargeLSTM_mapping.pkl', 'rb'))


In [36]:
# Make predictions
for k in range(n_chars_to_predict):
    # encode the characters as integers
    encoded = [mapping[char] for char in seed_text]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
    encoded = to_categorical(encoded, num_classes=len(mapping))
    # predict character
    yhat = model.predict_classes(encoded, verbose=0)
    
    # reverse map integer to character
    for char, index in mapping.items():
        if index == yhat:
            break
    seed_text += char

print(seed_text)

it was the best of times it was the worst of times it was the age of wisdom it was the age of foolishness it was the epoch of belief it was the epoch of incredulity it was the season of light it was the season of darkness it was the spring of hope it was the winter of despairwe had everything before us we had nothing before us we were all going direct to heaven we were all going direct the other way in his answer was to deliver it to be the only one things to be a little angry and seeming to be sure that he was the family s confidence into the courtyard here and from the great gentleman who was not a sigh nor confiscation and settled more and more disturbed. with the bastille citizen defarge stood striking his wife and child he said with a cries from the prisoner s hand and his face which had been taken in a corner where he laid her hand upon his head against the wall on the table between t
